In [ ]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
np.random.seed(42)
torch.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
data_dir = 'data/hymenoptera_data'
image_datasets = {'train':dataset_train, 'val':dataset_val} #тут надо вставить датасеты

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes #это может не работать

In [ ]:
N_CLASSES = 10

model_ft = torchvision.models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, N_CLASSES)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
from train import train_model

model_ft = train_model(model_ft, device, dataloaders, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

In [ ]:
from visualize import visualize_model

visualize_model(model_ft, device, dataloaders, class_names)